# Nonlinear Elasticity

_Created by [Rubén Sánchez Fernández](https://github.com/rsanfer) on 31/10/2017_   
_Last edited by [Rubén Sánchez Fernández](https://github.com/rsanfer) on 16/11/2017_

Once completed the tutorial in linear elasticity, we can move on to non-linear problems. SU2 has been designed using a finite-deformation framework$^1$ to account for geometrical and material non-linearities. In this tutorial, we use the same problem definition as for the linear elastic case. First, we will cover the definition of geometrical and material non-linearities. Then, we will solve a problem with multiple material definitions.

## Non-linear problems with single material.

We can write the non-linear structural problem via the residual equation

$$\mathscr{S}(\mathbf{u}) = \mathbf{T}(\mathbf{u}) - \mathbf{F}_b - \mathbf{F}_{\Gamma}(\mathbf{u})$$

which has been obtained from the weak formulation of the structural problem defined using the principle of virtual work and discretized using FEM. The state variable $\mathbf{u}$ corresponds to the displacements of the structure with respect to its reference configuration $\mathbf{R}$. $\mathbf{T}(\mathbf{u})$ are the internal equivalent forces, $\mathbf{F}_b$ are the body forces and $\mathbf{F}_{\Gamma}(\mathbf{u})$ are the surface forces applied over the external boundaries of the structure. We can linearize the structural equation as

$$\mathscr{S}(\mathbf{u}) + \frac{\partial \mathscr{S}(\mathbf{u})}{\partial \mathbf{u}} \Delta \mathbf{u} = \mathbf{0}$$

where the tangent matrix is $\mathbf{K} = \partial \mathscr{S}(\mathbf{u})/\partial \mathbf{u}$. The linearized problem is solved using a Newton method until convergence. Further description of the approach adopted in SU2 can be found in our recent [paper](https://doi.org/10.1002/nme.5700), published in the International Journal for Numerical Methods in Engineering$^2$.

We start the tutorial by definining the structural properties. In this case, we will solve a geometrically non-linear problem, with a Neo-Hookean material model. 

```
GEOMETRIC_CONDITIONS= LARGE_DEFORMATIONS
MATERIAL_MODEL= NEO_HOOKEAN
```

We adopt a plain strain formulation for the 2D problem, and reduce the stiffness of the cantilever by 2 orders of magnitude for it to undergo large deformations,

```
ELASTICITY_MODULUS=5E7
POISSON_RATIO=0.35
FORMULATION_ELASTICITY_2D = PLANE_STRAIN
```

Now, we define the Newton-Raphson strategy to solve the non-linear problem. We set the solution method and maximum number of subiterations using

```
NONLINEAR_FEM_SOLUTION_METHOD = NEWTON_RAPHSON
NONLINEAR_FEM_INT_ITER = 15
```

Then, the convergence criteria is set. The default criteria will evaluate the norm of the displacement vector (UTOL), the residual vector (RTOL) and the energy (ETOL), relative to the norm in the iteration 0. We set the convergence values to -8.0, which is the log<sub>10</sub> of the previous relative magnitudes.

```
RESIDUAL_FEM_UTOL = -8.0
RESIDUAL_FEM_RTOL = -8.0
RESIDUAL_FEM_ETOL = -8.0
```

Finally, we define the boundary conditions. In this case, the load applied to the ```left``` boundary will be an uniform, _follower_ pressure, that is, its direction will change with the changes in the normals of the surface. This is done as

```
MARKER_CLAMPED = ( clamped )
MARKER_PRESSURE = ( left, 1E3, upper, 0, right, 0)
```

#### Running the code

You can download the config and mesh files [here INSERT LINK](https://github.com/rsanfer/su2fsi). Execute the code with the standard command

```
SU2_CFD config.cfg
```

which will show the following convergence history:

```
 IntIter ExtIter     Res[UTOL]     Res[RTOL]     Res[ETOL]    VMS(Total)
       0       0      0.000000      0.000000      0.000000    1.1259e+06
       1       0     -0.943160      3.665996      2.044132    1.1224e+06
       2       0     -2.157899      2.289985     -0.745163    1.1053e+06
       3       0     -1.641303     -0.431490     -3.286888    1.1229e+06
       4       0     -3.092087      0.495168     -4.319279    1.1222e+06
       5       0     -3.851123     -2.418643     -7.658504    1.1221e+06
       6       0     -5.423121     -3.853135    -10.794810    1.1221e+06
       7       0     -7.022908     -6.063797    -13.996210    1.1221e+06
       8       0     -8.625487     -7.665757    -17.201359    1.1221e+06
       9       0    -10.227894     -8.696245    -20.405976    1.1221e+06
```

The code is stopped as soon as the values of ```Res[UTOL]```, ```Res[RTOL]``` and ```Res[ETOL]``` are below the convergence criteria set in the config file. The displacement field is shown in Fig. 1.

![Fig. 1](img/Fig1.png)

#### Increasing the load

The magnitude of the residual can limit the convergence of the solver, particularly for those cases in which the problem is very non-linear. Say, for example, that we multiply the load in the ```left``` boundary by 4, using

```
MARKER_PRESSURE = ( left, 4E3, upper, 0, right, 0)
```

Running SU2 now would result in the divergence of the solver,

```
 IntIter ExtIter     Res[UTOL]     Res[RTOL]     Res[ETOL]      VMS(Max)
       0       0      0.000000      0.000000      0.000000    4.5753e+06
       1       0     -0.154906      4.027944      3.085906    4.4618e+06
       2       0     -0.024596      4.057835      2.484297    3.7183e+09
       3       0          -nan           nan          -nan    0.0000e+00

```

due to a large imbalance between the internal and external loads. This can be also be tested by adding the command

```
RESIDUAL_CRITERIA_FEM = ABSOLUTE
```

to the config file, which will output the _absolute_ values of UTOL, RTOL and ETOL,

```
 IntIter ExtIter   Res[UTOL-A]   Res[RTOL-A]   Res[ETOL-A]      VMS(Max)
       0       0      0.000000      0.000000      0.000000    4.5753e+06
       1       0     -1.130461      4.628370      2.207826    4.4618e+06
       2       0     -0.523190      4.658260      1.606217    3.7183e+09
       3       0          -nan           nan          -nan    0.0000e+00
```

where it can be observed that the order of magnitude of the residual RTOL grows above 10<sup>4</sup>. To solve this problem, we have incorporated an incremental approach to SU2, that applies the load in linear steps,

$$\mathbf{F}_i = \frac{i}{N}\mathbf{F}$$

where $i$ is the step and $N$ the maximum number of increments allowed. To apply the load in increments, we need to use the following commands

```
INCREMENTAL_LOAD = YES
NUMBER_INCREMENTS = 25
INCREMENTAL_CRITERIA = (2.0, 2.0, 2.0)
```

where $N$ = 25, and the criteria to apply the increments is that ```Res[UTOL-A] > 2.0```, ```Res[RTOL-A] > 2.0``` or ```Res[ETOL-A] > 2.0```. If we now run the code,

```
 IntIter ExtIter   Res[UTOL-A]   Res[RTOL-A]   Res[ETOL-A]      VMS(Max)
       0       0      0.000000      0.000000      0.000000    4.5753e+06
       1       0     -1.130461      4.628370      2.207826    4.4618e+06

-- Incremental load: increment 1 ----------------------------------------
       0       0      0.000000      0.000000      0.000000    1.7944e+05
       1       0     -4.005792      2.096002     -3.207727    1.7948e+05
       2       0     -5.555428     -0.881184     -9.113457    1.7949e+05
       3       0     -7.611026     -4.228317    -14.254076    1.7949e+05
       4       0    -10.343697     -7.531347    -19.528441    1.7949e+05
       5       0    -13.137478     -8.798878    -24.948059    1.7949e+05

```

given that ```4.628370 > 2.0``` and ```2.207826 > 2.0```, the load is applied in $\Delta \mathbf{F} = 1/25 \mathbf{F}$, completing the simulation in the 25th step,

```
-- Incremental load: increment 25 ----------------------------------------
       0       0      0.000000      0.000000      0.000000    4.2701e+06
       1       0     -3.417010      2.241142     -2.911125    4.2325e+06
       2       0     -2.802795     -0.383540     -4.853108    4.1931e+06
       3       0     -3.461966      1.059169     -5.203155    4.2102e+06
       4       0     -4.053497     -0.412173     -7.024221    4.2103e+06
       5       0     -4.505028     -1.211625     -8.188042    4.2090e+06
       6       0     -5.044901     -2.434771     -9.288963    4.2093e+06
       7       0     -5.572267     -3.418852    -10.345812    4.2092e+06
       8       0     -6.099258     -4.140827    -11.400141    4.2093e+06
       9       0     -6.625915     -4.693663    -12.453409    4.2093e+06
      10       0     -7.152645     -5.224055    -13.506885    4.2093e+06
      11       0     -7.679350     -5.750728    -14.560291    4.2093e+06
      12       0     -8.206063     -6.277519    -15.613718    4.2093e+06
      13       0     -8.732773     -6.804167    -16.667139    4.2093e+06
      14       0     -9.259485     -7.330331    -17.720561    4.2093e+06
```

The displacement field is now

![Fig. 2](img/Fig2.png)

#### Running dynamic non-linear problems

As for the linear solver, we can run the non-linear structural solver in dynamic conditions. We add the sine load coefficient to the pressure marker,

```
SINE_LOAD= YES
SIN_LOAD_COEFF=(1.0, 2.0, 4.7124)
```

and incorporate the dynamic simulation commands,

```
DYNAMIC_ANALYSIS= YES

DYN_TIMESTEP= 0.01
DYN_TIME= 5.0
EXT_ITER=501

TIME_DISCRE_FEA= NEWMARK_IMPLICIT
NEWMARK_BETA=0.2601
NEWMARK_GAMMA=0.52
```

In this case, as the simulation will likely take longer, we can run it in parallel. Compile the software with [parallel support](https://github.com/su2code/SU2/wiki/Parallel-Build). We will need to set the restart file name, as it will be the only output of the code when run in parallel,

```
RESTART_STRUCTURE_FILENAME= restart_cantilever.dat
```

Then, we will use our mpi executable (mpirun/mpiexec) to run the simulation

```
mpirun -n 4 SU2_CFD config_dyn.cfg
```

In order to reconstruct the vtk files, we can use

```
SU2_SOL config_dyn.cfg
```

However, for this to work we need to set

```
SOLUTION_STRUCTURE_FILENAME= restart_cantilever.dat
```

so the code uses the restart files just generated to reconstruct the solution fields. The corresponding config file can be found here [here INSERT LINK](https://github.com/rsanfer/su2fsi)

## Non-linear problems with multiple material properties.

In this section, we will discretize the cantilever into four regions, R0, R1, R2 and R3,

![Fig. 3](img/Fig3.png)

in order to exemplify the ability of SU2 to deal with different material properties. The first thing required to deal with multiple materials, is to add the command

```
FEA_FILENAME=element_properties.dat
```

that defines the name of the element-based input file for material definition. This file has the format

```
INDEX MMOD MPROP ELPROP DV
0     0    0      0     0
1     0    0      0     0
...
249   0    0      0     0
250   0    1      0     0
...
499   0    1      0     0
500   0    2      0     0
...
749   0    2      0     0
750   0    3      0     0
...
999   0    3      0     0
```

where the fields, that must be separated by tabs, are:
- `INDEX` corresponds to the element ID number from the mesh file.
- `MMOD` is the material model. Using `0` sets the material model to be read from the command `MATERIAL_MODEL= NEO_HOOKEAN`. Alternatively, one could set different material models via material codes. At the time of writing, the material models available are:
  - 2: Neo-Hookean compressible
  - 4: Ideal DE model (2D only)
  - 5: Knowles stored-energy function$^3$
- `MPROP` sets the material properties. In this case, the Young's modulus and the Poisson ratio are:
  - `0`: $E$ = 50 GPa, $\nu$ = 0.4 
  - `1`: $E$ = 10 MPa, $\nu$ = 0.35
  - `2`: $E$ = 50 GPa, $\nu$ = 0.4
  - `3`: $E$ = 0.5 MPa, $\nu$ = 0.35
- `ELPROP` are the electric properties, discussed in another tutorial.
- `DV` are the identificators for design variables, discussed in another tutorial.

The regions R0, R1, R2 and R3 are assigned the `MPROP` `0`, `1`, `2` and `3` respectively, using the config options

```
ELASTICITY_MODULUS=(5E10, 1E7, 5E10, 5E5)
POISSON_RATIO=(0.4, 0.35, 0.4, 0.35)
```

Finally, given the flexibility of the regions R1 amd R3, an incremental approach with 10 increments is adopted. The solution is shown in 

![Fig. 4](img/Fig4.png)

where it can be observed how the flexible regions undergo large deformations, while the regions R0 and R2 remain virtually unaltered due to their high stiffness. The highlighted elements correspond to the lower layers of R1, R2 and R3.

##### References
$^1$ Bonet, J. and Wood, R.D. (2008), Nonlinear Continuum Mechanics for Finite Element Analysis, _Cambridge University Press_

$^2$ Sanchez, R., Albring, T., Palacios, R., Gauger, NR., Economon, T.D., Alonso, J.J., (2017) Coupled Adjoint-Based Sensitivities in Large-Displacement Fluid-Structure Interaction using Algorithmic Differentiation, _Int J Numer Meth Engng_

$^3$ Suchocki, C., (2011) A Finite Element Implementation of Knowles stored-energy function:
theory, coding and applications, _Archive of Mechanical Engineering, Vol. 58, pp. 319-346_

<dl>
<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons Licence" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by/4.0/">Creative Commons Attribution 4.0 International License</a>.
</dl>